In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as ne
import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
from train import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/models/scot/proportional_scot.yaml")
ETL_CONFIG = ETLConfig(name = "no_preprocessing", log1p = False, drop_unmatched_perts = True)
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configurations
config = Config(model_config=MODEL_CONFIG,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)


#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)


config.etl_config.pert_embedding = 'bioBERT'
config.etl_config.drop_unmatched_perts = True
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")


2025-02-02 08:04:02,823 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-02-02 08:04:02,825 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-02-02 08:04:02,827 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-02-02 08:04:02,829 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-02-02 08:04:02,830 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/kang.yaml
2025-02-02 08:04:02,832 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/essential_gene_knockouts_raw.yaml
2025-02-02 08:04:02,834 - INFO - Loading data catalogue from /orcd/data/omarabu/001/njwfish/omnicell/configs/catalogue/satija_IFNG_raw_INCOMPLET

Using device: cuda


2025-02-02 08:04:06,813 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-02-02 08:04:06,814 - INFO - Preprocessing training data
2025-02-02 08:04:06,816 - INFO - Using identity features for perturbations
2025-02-02 08:04:06,937 - INFO - Removing observations with perturbations not in the dataset as a column
2025-02-02 08:04:07,123 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
/orcd/data/omarabu/001/njwfish/omnicell/notebooks/../omnicell/data/loader.py:178: ImplicitModificationWarning: Setting element `.obsm['embedding']` of view, initializing view as actual.
  adata.obsm["embedding"] = adata.X.toarray().astype('float32')
2025-02-02 08:04:43,549 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [ ]:
import numpy as np
from omnicell.models.datamodules import get_dataloader
dset, ns, dl = get_dataloader(adata, pert_ids=np.array(adata.obs[PERT_KEY].values), pert_map=pert_rep_map, collate='ot')


In [3]:
import pickle
from omnicell.models.distribute_shift import (
    get_proportional_weighted_dist,
    sample_multinomial_batch,
    sample_pert
)

batch_size = 512
num_ctrls = dset.source['k562'].shape[0]

for i in range(0, num_ctrls, batch_size):
    source_batch = {} 
    synthetic_counterfactual_batch = {}
    for stratum in dset.strata:
        source_batch[stratum] = dset.source[stratum][i:i+512]
        synthetic_counterfactual_batch[stratum] = {}

        for j, pert in enumerate(dset.unique_pert_ids):
            if j % 100 == 0:
                print(f"{i} / {num_ctrls}, {j} / {len(dset.unique_pert_ids)}")
            X_ctrl = source_batch[stratum]
            X_pert = dset.target[stratum][pert]
            mean_ctrl = X_ctrl.mean(axis=0)
            mean_pert = X_pert.mean(axis=0)
            
            mean_shift = mean_pert - mean_ctrl
            logger.debug(f"Mean shift shape: {mean_shift.shape}")

            weighted_dist = get_proportional_weighted_dist(X_ctrl)
            
            preds = sample_pert(X_ctrl, weighted_dist, mean_shift, max_rejections=100)
            synthetic_counterfactual_batch[stratum][pert] = preds.astype(np.int16)
        
    # Save as before
    data_dict = {
        'synthetic_counterfactuals': synthetic_counterfactual_batch,
        'source': source_batch,
        'unique_pert_ids': dset.unique_pert_ids,
        'strata': dset.strata
    }

    with open(f'/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals_{i}.pkl', 'wb') as f:
        pickle.dump(data_dict, f)


0 / 10691, 0 / 1849


ValueError: sum(pvals[:-1]) > 1.0

In [8]:
import pickle
from omnicell.models.distribute_shift import (
    get_proportional_weighted_dist,
    sample_multinomial_batch,
    sample_pert
)

batch_size = 512
num_ctrls = dset.source['k562'].shape[0]

for i in range(0, num_ctrls, batch_size):
    source_batch = {} 
    synthetic_counterfactual_batch = {}
    for stratum in dset.strata:
        source_batch[stratum] = dset.source[stratum][i:i+512]
        synthetic_counterfactual_batch[stratum] = {}

        for j, pert in enumerate(dset.unique_pert_ids):
            if j % 100 == 0:
                print(f"{i} / {num_ctrls}, {j} / {len(dset.unique_pert_ids)}")
            X_ctrl = source_batch[stratum]
            X_pert = dset.target[stratum][pert]
            mean_ctrl = X_ctrl.mean(axis=0)
            mean_pert = X_pert.mean(axis=0)
            
            mean_shift = mean_pert - mean_ctrl
            logger.debug(f"Mean shift shape: {mean_shift.shape}")

            weighted_dist = get_proportional_weighted_dist(X_ctrl)
            
            preds = sample_pert(X_ctrl, weighted_dist, mean_shift, max_rejections=100)
            synthetic_counterfactual_batch[stratum][pert] = preds.astype(np.int16)
        
    # Save as before
    data_dict = {
        'synthetic_counterfactuals': synthetic_counterfactual_batch,
        'source': source_batch,
        'unique_pert_ids': dset.unique_pert_ids,
        'strata': dset.strata
    }

    with open(f'/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals_{i}.pkl', 'wb') as f:
        pickle.dump(data_dict, f)


0 / 20, 0 / 1849
0 / 20, 100 / 1849
0 / 20, 200 / 1849
0 / 20, 300 / 1849
0 / 20, 400 / 1849
0 / 20, 500 / 1849
0 / 20, 600 / 1849
0 / 20, 700 / 1849
0 / 20, 800 / 1849
0 / 20, 900 / 1849
0 / 20, 1000 / 1849
0 / 20, 1100 / 1849
0 / 20, 1200 / 1849
0 / 20, 1300 / 1849
0 / 20, 1400 / 1849
0 / 20, 1500 / 1849
0 / 20, 1600 / 1849
0 / 20, 1700 / 1849
0 / 20, 1800 / 1849
512 / 20, 0 / 1849
512 / 20, 100 / 1849
512 / 20, 200 / 1849
512 / 20, 300 / 1849
512 / 20, 400 / 1849
512 / 20, 500 / 1849
512 / 20, 600 / 1849
512 / 20, 700 / 1849
512 / 20, 800 / 1849


KeyboardInterrupt: 

In [ ]:
import pickle
from omnicell.models.distribute_shift import sample_pert, get_proportional_weighted_dist

batch_size = 512
num_batches = dset.source['k562'].shape[0] // batch_size

for i in range(0, dset.source['k562'].shape[0], batch_size):
    source_batch = {} 
    synthetic_counterfactual_batch = {}
    for stratum in dset.strata:
        source_batch[stratum] = dset.source[stratum][i:i+512]
        synthetic_counterfactual_batch[stratum] = {}

        for j, pert in enumerate(dset.unique_pert_ids):
            
            print(f"{i} / {num_batches}, {j} / {len(dset.unique_pert_ids)}")
            X_ctrl = source_batch[stratum]
            X_pert = dset.target[stratum][pert]
            mean_ctrl = X_ctrl.mean(axis=0)
            mean_pert = X_pert.mean(axis=0)
            
            mean_shift = mean_pert - mean_ctrl
            logger.debug(f"Mean shift shape: {mean_shift.shape}")

            weighted_dist = get_proportional_weighted_dist(X_ctrl)
            
            preds = sample_pert(X_ctrl, weighted_dist, mean_shift, max_rejections=100)
            synthetic_counterfactual_batch[stratum][pert] = preds.astype(np.int16)
        
    # Save as before
    data_dict = {
        'synthetic_counterfactuals': synthetic_counterfactual_batch,
        'source': source_batch,
        'unique_pert_ids': dset.unique_pert_ids,
        'strata': dset.strata
    }

    with open(f'/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals_{i}.pkl', 'wb') as f:
        pickle.dump(data_dict, f)

In [24]:
# convert to synthetic_counterfactuals, source, and target to np.int16, also convert to sparse
# from scipy import sparse

# Convert synthetic counterfactuals to sparse
synthetic_counterfactuals = {
    stratum: {
        pert: preds.astype(np.int16)
        for pert, preds in synthetic_counterfactuals[stratum].items()
    } 
    for stratum in synthetic_counterfactuals
}

# Convert source to sparse
dset.source = {
    stratum: source.astype(np.int16)
    for stratum, source in dset.source.items()
}

# Convert target to sparse
dset.target = {
    stratum: {
        pert: preds.astype(np.int16)
        for pert, preds in dset.target[stratum].items()
    } 
    for stratum in dset.target
}


In [ ]:
# Saving
import pickle
# Save as before
data_dict = {
    'synthetic_counterfactuals': synthetic_counterfactuals,
    'source': dset.source,
    'target': dset.target,
    'unique_pert_ids': dset.unique_pert_ids,
    'strata': dset.strata
}
with open('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals.pkl', 'wb') as f:
    pickle.dump(data_dict, f)




In [1]:
# Loading
import pickle
with open('/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/proportional_scot/synthetic_counterfactuals.pkl', 'rb') as f:
    data_dict = pickle.load(f)
    
synthetic_counterfactuals = data_dict['synthetic_counterfactuals']
source = data_dict['source']
target = data_dict['target']
unique_pert_ids = data_dict['unique_pert_ids']
strata = data_dict['strata']